In [1]:
import pandas as pd
import xgboost as xgb
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
import pickle


In [2]:
from sklearn import datasets


In [3]:
iris = datasets.load_iris()
X = iris.data[:, :3]  # we only take the first two features.
df = pd.DataFrame(X,columns = ["sepal length","sepal width", "petal length"])
df['classIndex'] = iris.target

In [4]:
from sklearn.model_selection import train_test_split
# Split to train/test
training_df, test_df = train_test_split(df)

In [12]:
training_df = training_df[(training_df['classIndex']==0)|(training_df['classIndex']==1)]
test_df = test_df[(test_df['classIndex']==0)|(test_df['classIndex']==1)]

In [5]:
X_train = training_df.drop(columns = ['classIndex'])
y_train = training_df.classIndex

In [6]:
X_test = test_df.drop(columns = ['classIndex'])
y_test = test_df.classIndex

In [7]:
xgb_cl = xgb.XGBClassifier()
xgb_cl.fit(X_train, y_train)

/Users/mosal/miniconda3/envs/mlflow/lib/python3.10/site-packages/xgboost/data.py:299: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/mosal/miniconda3/envs/mlflow/lib/python3.10/site-packages/xgboost/data.py:301: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/Users/mosal/miniconda3/envs/mlflow/lib/python3.10/site-packages/xgboost/data.py:332: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/Users/mosal/miniconda3/envs/mlflow/lib/python3.10/site-packages/xgboost/data.py:323: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) i

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [9]:
import pickle

# Save the model to a file
pickle.dump(xgb_cl, open("../models/flow_mosal2024.pkl", "wb"))

In [8]:
# all

# in sample
preds_prob_train = xgb_cl.predict_proba(X_train)[::,1] # get probability of 1 - out of sample

# out of sample
preds_prob_test = xgb_cl.predict_proba(X_test)[::,1] # get probability of 1 - out of sample

/Users/mosal/miniconda3/envs/mlflow/lib/python3.10/site-packages/xgboost/data.py:299: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/mosal/miniconda3/envs/mlflow/lib/python3.10/site-packages/xgboost/data.py:301: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/Users/mosal/miniconda3/envs/mlflow/lib/python3.10/site-packages/xgboost/data.py:332: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/Users/mosal/miniconda3/envs/mlflow/lib/python3.10/site-packages/xgboost/data.py:323: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) i

In [21]:
# tune optimal probability threshold
threshold = [x*0.001 for x in range(1000)]
score = [metrics.f1_score(y_test, preds_prob_test > t) for t in threshold]
optimal_threshold = threshold[np.argmax(score)]

np.max(score), threshold[np.argmax(score)]


(1.0, 0.026000000000000002)

In [22]:
preds_test_optimal = [int(x) for x in preds_prob_test > optimal_threshold]
# out of sample
print(metrics.f1_score(y_test, preds_test_optimal))
print(metrics.confusion_matrix(y_test, preds_test_optimal))
print(metrics.roc_auc_score(y_test, preds_prob_test))

1.0
[[13  0]
 [ 0 11]]
1.0


In [24]:
model_path = 'models/flow_mosal.pkl'

with open(model_path, 'wb') as fid:
    pickle.dump(xgb_cl, fid)